In [ ]:
pip install QuantLib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 29.4 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import mean_squared_error


# 总览


In [ ]:

import QuantLib as ql
import pandas as pd


todaysDate = ql.Date(11, 5, 2017)
ql.Settings.instance().evaluationDate = todaysDate


interactive = "get_ipython" in globals()

spot_price = 1
K=2.5
q=0.0
r=0.001
sigma=2
maturity_date = ql.Date(11, 8, 2017)#到期日
  # %%
exercise = ql.AmericanExercise(todaysDate, maturity_date)
payoff = ql.PlainVanillaPayoff(ql.Option.Call, K)

  # %%
option = ql.VanillaOption(payoff, exercise)

  # %% [markdown]
  # ### Market data

  # %%


underlying = ql.SimpleQuote(spot_price)
dividendYield = ql.FlatForward(todaysDate, q, ql.Actual365Fixed())
volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), sigma, ql.Actual365Fixed())
riskFreeRate = ql.FlatForward(todaysDate, r, ql.Actual365Fixed())


  # %%
process = ql.BlackScholesMertonProcess(
      ql.QuoteHandle(underlying),
      ql.YieldTermStructureHandle(dividendYield),
      ql.YieldTermStructureHandle(riskFreeRate),
      ql.BlackVolTermStructureHandle(volatility),
  )

  # %% [markdown]
  # ### Pricing
  #
  # We'll collect tuples of method name, option value, and estimated error from the analytic formula.

  # %%
results = []

  # %% [markdown]
  # #### Analytic approximations

  # %%
option.setPricingEngine(ql.BaroneAdesiWhaleyApproximationEngine(process))
results.append(("Barone-Adesi-Whaley", option.NPV()))

  # %%
option.setPricingEngine(ql.BjerksundStenslandApproximationEngine(process))
results.append(("Bjerksund-Stensland", option.NPV()))

  # %% [markdown]
  # #### Finite-difference method

  # %%
timeSteps = 201
gridPoints = 200

  # %%
option.setPricingEngine(ql.FdBlackScholesVanillaEngine(process, timeSteps, gridPoints))
results.append(("finite differences", option.NPV()))


  # %% [markdown]
  # #### Li, M. QD+ American engine

  # %%
option.setPricingEngine(ql.QdPlusAmericanEngine(process))
results.append(("QD+", option.NPV()))


  # %% [markdown]
  # #### Leif Andersen, Mark Lake and Dimitri Offengenden high performance American engine

  # %%
option.setPricingEngine(ql.QdFpAmericanEngine(
      process, ql.QdFpAmericanEngine.accurateScheme()))
results.append(("QD+ fixed point", option.NPV()))


  # %% [markdown]
  # #### Binomial method

  # %%
timeSteps =100

  # %%
for tree in ["JR", "CRR", "EQP", "Trigeorgis", "Tian", "LR", "Joshi4"]:
      option.setPricingEngine(ql.BinomialVanillaEngine(process, tree, timeSteps))
      results.append(("Binomial (%s)" % tree, option.NPV()))

  # %% [markdown]
  # ### Results

  # %%
df = pd.DataFrame(results, columns=["Method", "Option value"])
df.style.hide_index()

  # %% [markdown]
  # The following displays the results when this is run as a Python script (in which case the cell above is not displayed).

  # %%
if not interactive:
      print(df)
df

<ipython-input-3-39c12c9e27d7>:108: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  df.style.hide_index()


,Method,Option value
0,Barone-Adesi-Whaley,0.144287
1,Bjerksund-Stensland,0.144287
2,finite differences,0.144263
3,QD+,0.144287
4,QD+ fixed point,0.144287
5,Binomial (JR),0.142195
6,Binomial (CRR),0.144723
7,Binomial (EQP),0.160094
8,Binomial (Trigeorgis),0.145619
9,Binomial (Tian),0.144912


# BAW


In [ ]:
#Barone-Adesi-Whaley
import QuantLib as ql
def Americanop(m,T,r,sigma,q):
  todaysDate = ql.Date(11, 5, 2017)
  ql.Settings.instance().evaluationDate = todaysDate
  K = m
  spot_price = 1
  maturity_date = todaysDate + ql.Period(f"{int(T*365)}d")#到期日

  exercise = ql.AmericanExercise(todaysDate, maturity_date)
  payoff = ql.PlainVanillaPayoff(ql.Option.Call, K)
  # %%
  option = ql.VanillaOption(payoff, exercise)
  underlying = ql.SimpleQuote(spot_price)
  dividendYield = ql.FlatForward(todaysDate, q, ql.Actual365Fixed())
  volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), sigma, ql.Actual365Fixed())
  riskFreeRate = ql.FlatForward(todaysDate, r, ql.Actual365Fixed())
  # %%
  process = ql.BlackScholesMertonProcess(
      ql.QuoteHandle(underlying),
      ql.YieldTermStructureHandle(dividendYield),
      ql.YieldTermStructureHandle(riskFreeRate),
      ql.BlackVolTermStructureHandle(volatility),
  )
  option.setPricingEngine(ql.BaroneAdesiWhaleyApproximationEngine(process))
  return option.NPV()

print(Americanop(2.5,92/365,0.001,2,0.0))


0.14428730291973105


# CRR


In [ ]:
#crr
import QuantLib as ql
def ameri_crr(m,T,r,sigma,q):
  todaysDate = ql.Date(11, 5, 2017)
  ql.Settings.instance().evaluationDate = todaysDate
  K = m
  spot_price = 1


  #timeSteps = 100
  timeSteps = max(1000, int(4000 * T))
  maturity_date = todaysDate + ql.Period(f"{int(T*365)}d")#到期日

  exercise = ql.AmericanExercise(todaysDate, maturity_date)
  payoff = ql.PlainVanillaPayoff(ql.Option.Call, K)
  # %%
  option = ql.VanillaOption(payoff, exercise)
  underlying = ql.SimpleQuote(spot_price)
  dividendYield = ql.FlatForward(todaysDate, q, ql.Actual365Fixed())
  volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), sigma, ql.Actual365Fixed())
  riskFreeRate = ql.FlatForward(todaysDate, r, ql.Actual365Fixed())
  # %%
  process = ql.BlackScholesMertonProcess(
      ql.QuoteHandle(underlying),
      ql.YieldTermStructureHandle(dividendYield),
      ql.YieldTermStructureHandle(riskFreeRate),
      ql.BlackVolTermStructureHandle(volatility),
  )
  option.setPricingEngine(ql.BinomialVanillaEngine(process, "CRR", timeSteps))
  return option.NPV()

print(ameri_crr(2.5,92/365,0.001,2,0.0))


0.14433108541939632


# BBS + BBSR


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as si
from sklearn.metrics import mean_squared_error
import time

def BSM(Put_Call, S_0, X, rfr, vol, t,q):
    d1 = (np.log(X/S_0) + (rfr + 0.5 * vol ** 2) * t -q) / (vol * np.sqrt(t))
    d2 = (np.log(X/S_0) + (rfr - 0.5 * vol ** 2) * t-q) / (vol * np.sqrt(t))

    if Put_Call == "C":
        opt_price = S_0*si.norm.cdf(d1) - X*np.exp(-rfr*t)*si.norm.cdf(d2)
    elif Put_Call == "P":
        opt_price = X*np.exp(-rfr*t)*si.norm.cdf(-d2) - S_0*si.norm.cdf(-d1)

    return opt_price
def BBS(Put_Call, n, S_0, X, t,rfr, vol,q):  #rfr is risk free rate, vol is volatility
    deltaT = t/n
    u = np.exp(vol*np.sqrt(deltaT))
    d = 1./u
    R = np.exp(rfr*deltaT)
    p = (R-d)/(u-d)
    q = 1-p

    # simulating the underlying price paths
    S = np.zeros((n+1,n+1))
    S[0,0] = S_0
    for i in range(1,n+1):
        S[i,0] = S[i-1,0]*u
        for j in range(1,i+1):
            S[i,j] = S[i-1,j-1]*d

    # option value at final node
    V = np.zeros((n+1,n+1)) # V[i,j] is the option value at node (i,j)
    for j in range(n+1):
        if Put_Call=="C":
            V[n,j] = max(0, S[n,j]-X)
        elif Put_Call=="P":
            V[n,j] = max(0, X-S[n,j])
    #for j in range(n):
    #    V[n-1,j] = BSM(Put_Call, S[n-1,j], X, rfr, vol, t/n,q)


    for i in range(n-2,-1,-1):
        for j in range(i+1):
            if Put_Call=="P":
                V[i,j] = max(0, X-S[i,j], 1/R*(p*V[i+1,j]+q*V[i+1,j+1]))
            elif Put_Call=="C":
                V[i,j] = max(0, S[i,j]-X, 1/R*(p*V[i+1,j]+q*V[i+1,j+1]))
        opt_price = V[0,0]

    return opt_price
def BBSR(Put_Call, n, S_0, X, t, rfr, vol,q):
    #opt_price = 2*BBS(Put_Call, n, S_0, X,t, rfr, vol,q) - BBS(Put_Call, int(n/2), S_0, X, t,rfr, vol,q)
    opt_price = 8/3*BBS(Put_Call,n,S_0, X,t, rfr, vol,q)- 2*BBS(Put_Call,int(n/2),S_0, X,t, rfr, vol,q) + 1/3*BBS(Put_Call,int(n/4),S_0, X,t, rfr, vol,q)

    return opt_price
S_0 = 1
rfr = 0.001
vol = 2
X = 2.5
t = 92/365
q=0

v_BBSR = BBSR("C", 1000, 1, X,t, rfr, vol,q)

v_BBS = BBS("C", 1000, 1, X, t, rfr, vol,q)

print(v_BBSR, v_BBS)


SyntaxError: ignored



# FDE

In [ ]:
#FDE
import QuantLib as ql
def amer_fde(m,T,r,sigma,q):
  #%%
  todaysDate = ql.Date(11, 5, 2017)
  maturity_date = todaysDate + ql.Period(f"{int(T*365)}d")#到期日

  #timeSteps = 800  #801
  timeSteps = max(1000, int(4000 * T))
  gridPoints = max(1000, int(1000 * T))#timeSteps-1  #801-1


  #%%

  #initial
  ql.Settings.instance().evaluationDate = todaysDate
  K = m
  spot_price = 1


  exercise = ql.AmericanExercise(todaysDate, maturity_date)
  payoff = ql.PlainVanillaPayoff(ql.Option.Call, K)
  # %%
  option = ql.VanillaOption(payoff, exercise)
  underlying = ql.SimpleQuote(spot_price)
  dividendYield = ql.FlatForward(todaysDate, q, ql.Actual365Fixed())
  volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), sigma, ql.Actual365Fixed())
  riskFreeRate = ql.FlatForward(todaysDate, r, ql.Actual365Fixed())
  # %%
  process = ql.BlackScholesMertonProcess(
      ql.QuoteHandle(underlying),
      ql.YieldTermStructureHandle(dividendYield),
      ql.YieldTermStructureHandle(riskFreeRate),
      ql.BlackVolTermStructureHandle(volatility),
  )


    # %%
  option.setPricingEngine(ql.FdBlackScholesVanillaEngine(process, timeSteps, gridPoints,int(timeSteps * 0.2)))
  return option.NPV()

print(amer_fde(2.5,92/365,0.001,2,0.0))


0.14428370555828793


In [ ]:
#FDE
import QuantLib as ql
def amer_fde2(m,T,r,sigma,q):
  #%%
  todaysDate = ql.Date(11, 5, 2017)
  maturity_date = todaysDate + ql.Period(f"{int(T*365)}d")#到期日

  #timeSteps = 800  #801
  timeSteps = max(1000, int(4000 * T))
  gridPoints = max(1000, int(1000 * T))#timeSteps-1  #801-1

  #%%

  #initial
  ql.Settings.instance().evaluationDate = todaysDate
  K = m
  spot_price = 1


  exercise = ql.AmericanExercise(todaysDate, maturity_date)
  payoff = ql.PlainVanillaPayoff(ql.Option.Call, K)
  # %%
  option = ql.VanillaOption(payoff, exercise)
  underlying = ql.SimpleQuote(spot_price)
  dividendYield = ql.FlatForward(todaysDate, q, ql.Actual365Fixed())
  volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), sigma, ql.Actual365Fixed())
  riskFreeRate = ql.FlatForward(todaysDate, r, ql.Actual365Fixed())
  # %%
  process = ql.BlackScholesMertonProcess(
      ql.QuoteHandle(underlying),
      ql.YieldTermStructureHandle(dividendYield),
      ql.YieldTermStructureHandle(riskFreeRate),
      ql.BlackVolTermStructureHandle(volatility),
  )


    # %%
  option.setPricingEngine(ql.FdBlackScholesVanillaEngine(process, timeSteps, gridPoints))
  return option.NPV()

print(amer_fde(2.5,92/365,0.001,2,0.0))


0.14428370555828793


# MC



In [ ]:
#PseudoRandom
import QuantLib as ql
mcSeed = 100
method = 'MC (Longstaff Schwartz)'

def ameri_mc(m,T,r,sigma,q):
  todaysDate = ql.Date(11, 5, 2017)
  ql.Settings.instance().evaluationDate = todaysDate
  K = m
  spot_price = 1


  #timeSteps = 100
  timeSteps = max(100, int(200 * T))
  maturity_date = todaysDate + ql.Period(f"{int(T*365)}d")#到期日

  exercise = ql.AmericanExercise(todaysDate, maturity_date)
  payoff = ql.PlainVanillaPayoff(ql.Option.Call, K)
  # %%
  option = ql.VanillaOption(payoff, exercise)
  underlying = ql.SimpleQuote(spot_price)
  dividendYield = ql.FlatForward(todaysDate, q, ql.Actual365Fixed())
  volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), sigma, ql.Actual365Fixed())
  riskFreeRate = ql.FlatForward(todaysDate, r, ql.Actual365Fixed())
  # %%
  process = ql.BlackScholesMertonProcess(
      ql.QuoteHandle(underlying),
      ql.YieldTermStructureHandle(dividendYield),
      ql.YieldTermStructureHandle(riskFreeRate),
      ql.BlackVolTermStructureHandle(volatility),
  )
  rng='pseudorandom'
  #rng='lowdiscrepancy'
  mcengine3 = ql.MCAmericanEngine(
    process,
    rng,
    timeSteps,
    antitheticVariate=True,
    #nCalibrationSamples=5000,
    #requiredTolerance=0.02,
    requiredSamples=10000,
    seed=mcSeed)
  option.setPricingEngine(mcengine3)

  return option.NPV()

print(ameri_crr(2.5,92/365,0.001,2,0.0))


0.1446640497614923


# BBS(误差超大—）


In [ ]:
#crr
import QuantLib as ql
def BBS1(m,T,r,sigma,q,ts):
  todaysDate = ql.Date(11, 5, 2017)
  ql.Settings.instance().evaluationDate = todaysDate
  K = m
  spot_price = 1
  timeSteps = 2


  #timeSteps = 100
  #timeSteps = max(1000, int(4000 * T))
  if ts == '0':
      timeSteps = int(max(1000, int(4000 * T))/2)
  if ts == '1':
      timeSteps = max(1000, int(4000 * T))

  maturity_date = todaysDate + ql.Period(f"{int(T*365)}d")#到期日

  exercise = ql.AmericanExercise(todaysDate, maturity_date)
  payoff = ql.PlainVanillaPayoff(ql.Option.Call, K)
  # %%
  option = ql.VanillaOption(payoff, exercise)
  underlying = ql.SimpleQuote(spot_price)
  dividendYield = ql.FlatForward(todaysDate, q, ql.Actual365Fixed())
  volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), sigma, ql.Actual365Fixed())
  riskFreeRate = ql.FlatForward(todaysDate, r, ql.Actual365Fixed())
  # %%
  process = ql.BlackScholesMertonProcess(
      ql.QuoteHandle(underlying),
      ql.YieldTermStructureHandle(dividendYield),
      ql.YieldTermStructureHandle(riskFreeRate),
      ql.BlackVolTermStructureHandle(volatility),
  )
  option.setPricingEngine(ql.BinomialVanillaEngine(process, 'CRR', timeSteps))
  return option.NPV()


print(BBS1(2.5,92/365,0.001,2,0.0,'0'))


0.1442754982209777


In [ ]:
def BBSR1( X, t, rfr, vol,q):
  opt_price = 2*BBS1( X,t, rfr, vol,q,'1') - BBS1(X, t,rfr, vol,q,'0')
    #opt_price = 8/3*BBS1(Put_Call,n,S_0, X,t, rfr, vol,q)- 2*BBS1(Put_Call,int(n/2),S_0, X,t, rfr, vol,q) + 1/3*BBS1(Put_Call,int(n/4),S_0, X,t, rfr, vol,q)
  return opt_price
print(BBSR1(2.5,92/365,0.001,2,0.0))

0.14438667261781493


# 欧式


In [ ]:
#
def eurocall_ql(strike_price,T,risk_free_rate,volatility,dividend_rate):
  spot_price=1
  option_type = ql.Option.Call

  day_count = ql.Actual365Fixed()
  calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)

  calculation_date = ql.Date(11, 5, 2017)#起始日期
  ql.Settings.instance().evaluationDate = calculation_date
  maturity_date = calculation_date + ql.Period(f"{int(T*365)}d")
  payoff = ql.PlainVanillaPayoff(option_type, strike_price)
  exercise = ql.EuropeanExercise(maturity_date)
  european_option = ql.VanillaOption(payoff, exercise)

  spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
  flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count))

  dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count))
  flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, volatility, day_count))

  bsm_process = ql.BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)

  european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
  return   european_option.NPV()


print(eurocall_ql(2.5,92/365,0.001,2,0.0))


0.14428730291973105


# 生成数据


In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
os.getcwd()
p = Path(".")

m = np.random.uniform(0.8,1.2,size = 10000)
T = np.array([4.0/365, 11.0/365, 18.0/365, 25.0/365, 32.0/365, 39.0/365, 46.0/365, 73.96/365, 100.96/365, 136.96/365, 164.96/365, 192.96/365, 255.96/365, 382.0/365, 437.96/365, 528.96/365, 619.96/365, 746.0/365])
#%%
#%%
r = np.random.uniform(0,0.08,size=1000)

#q = np.random.uniform(0,0,size=1000)
q = np.random.uniform(0,0.05,size=1000)

sigma = np.random.uniform(0.05,2,size=1000)



g_m = np.random.choice(m, 10000)
g_T = np.random.choice(T, 10000)
g_r = np.random.choice(r, 10000)
g_q = np.random.choice(q, 10000)
g_sigma =np.random.choice(sigma,10000)

eurocall =[]
amer_CRR =[]
amer_BAW =[]
amer_BBS =[]
amer_BBSR =[]
amer_FDE =[]
amer_FDE2 = []
amer_MC = []


#for i in range(2000):
#  amer_CRR.append(ameri_crr(g_m[i],g_T[i],g_r[i],g_sigma[i],g_q[i]))
#for i in range(2000):
#  amer_BAW.append(Americanop(g_m[i],g_T[i],g_r[i],g_sigma[i],g_q[i]))
#for i in range(2000):
#  amer_BBS.append(BBS1("C",1000,1,g_m[i],g_T[i],g_r[i],g_sigma[i],g_q[i]))
#for i in range(2000):
#  amer_BBSR.append(BBSR1("C",1000,1,g_m[i],g_T[i],g_r[i],g_sigma[i],g_q[i]))
for i in range(10000):
  amer_FDE.append(amer_fde(g_m[i],g_T[i],g_r[i],g_sigma[i],g_q[i]))
#for i in range(2000):
#  amer_FDE2.append(amer_fde2(g_m[i],g_T[i],g_r[i],g_sigma[i],g_q[i]))
#for i in range(2000):
# amer_FDE3.append(amer_fde3(g_m[i],g_T[i],g_r[i],g_sigma[i],g_q[i]))

#for i in range(2000):
#  amer_BBS.append(BBS1(g_m[i],g_T[i],g_r[i],g_sigma[i],g_q[i]))
#for i in range(2000):
#  amer_BBSR.append(BBSR1(g_m[i],g_T[i],g_r[i],g_sigma[i],g_q[i]))
#for i in range(2000):
#  amer_MC.append(ameri_mc(g_m[i],g_T[i],g_r[i],g_sigma[i],g_q[i]))

#for i in range(2000):
#  eurocall.append(eurocall_ql(g_m[i],g_T[i],g_r[i],g_sigma[i],g_q[i]))


g_eurocall = eurocall
g_amercall_crr = amer_CRR
g_amercall_baw = amer_BAW
g_amercall_fde = amer_FDE
g_amercall_bbs = amer_BBS
g_amercall_bbsr = amer_BBSR

g_amercall_fde2 = amer_FDE2
g_amercall_mc = amer_MC


generated_data = pd.DataFrame({
  'g_m':g_m,
  'g_T': g_T,
  'g_r':g_r,
  'g_q':g_q,
  'g_sigma':g_sigma,

# 'g_eurocall':g_eurocall,

# 'g_amercall_crr':g_amercall_crr,
#  'g_amercall_baw':g_amercall_baw,
  'g_amercall_fde':g_amercall_fde,
#  'g_amercall_fde2':g_amercall_fde2,
#  'g_amercall_mc':g_amercall_mc,

#  'g_amercall_bbs':g_amercall_bbs,
#  'g_amercall_bbsr':g_amercall_bbsr

  }).to_csv(p /"/content" / "generated_data.csv")

In [ ]:
import pandas as pd
data = pd.read_csv("/content/generated_data.csv")
data.head()

,Unnamed: 0,g_m,g_T,g_r,g_q,g_sigma,g_eurocall,g_amercall_fde,g_amercall_fde2
0,0,4.056438,1.698521,0.054303,0.0,0.263485,1.050735e-05,0.000011,0.000011
1,1,4.957985,0.701260,0.014452,0.0,0.084536,1.015390e-114,0.000000,0.000000
2,2,4.005761,1.046575,0.036741,0.0,0.317575,2.198618e-06,0.000002,0.000002
3,3,3.325164,0.528658,0.052374,0.0,1.243657,6.863528e-02,0.068636,0.068634
4,4,2.434647,1.199890,0.075319,0.0,0.244126,1.545958e-04,0.000155,0.000155


In [ ]:

rsme = []
#rsme.append(('CRR',np.sqrt(mean_squared_error(data.g_amercall_crr, data.g_eurocall))))
#rsme.append(('Barone-Adesi-Whaley',np.sqrt(mean_squared_error(data.g_amercall_baw, data.g_eurocall))))
rsme.append(('Crank-Nicolson Finite Differences',np.sqrt(mean_squared_error(data.g_amercall_fde, data.g_eurocall))))

rsme.append(('Implicit Finite Differences',np.sqrt(mean_squared_error(data.g_amercall_fde2, data.g_eurocall))))

#rsme.append(('BBS',np.sqrt(mean_squared_error(data.g_amercall_bbs, data.g_eurocall))))
#rsme.append(('BBSR',np.sqrt(mean_squared_error(data.g_amercall_bbsr, data.g_eurocall))))

df = pd.DataFrame(rsme, columns=["Method", "RMSE"])
df.style.hide_index()

Method,RMSE
Crank-Nicolson Finite Differences,0.000004
Implicit Finite Differences,0.000003
